# TFM - Predicción potenciales compradores de un ecommerce

Este trabajo, está principalmente enfocado a la creación de un modelo de predicción de potenciales compradores en un ecommerce real relacionado con el mundo empresarial.

Se trata de realizar una clasificación de visitantes a una web de venta online en función de los datos que se han almacenado de ellos y establecer su nivel potencial de compra.

Para ello se dispone de diferentes repositorios con datos históricos relativos a:
* características básicas del registro/lead: fecha registro, procedencia, marca de comprador, cuando compró.
* características identificativas anonimizadas de los mismos: email, ubicación geográfica, tipo de persona.
* log de sus movimientos/navegación en la web.
* información sobre los productos que ha comprado.

**Información relevante del negocio**

Los datos proceden de un ecommerce con sede en Colombia que se dedica a la venta de productos relacionados con la información de empresas del país: Informes Comerciales y módulos de información detallada sobre Datos Financieros, Prensa, Administradores, Incidencias, etc, Informes Sectoriales, Base de datos a medida, Productos de Marketing (mercadeo), Información de accionistas, Información de proveedores y clientes, etc.

Los usuarios potenciales compradores (personas en su nombre o representando a una empresa u otra entidad), llegan a la web del ecommerce por diferentes canales: directorios propios, webs de terceros, desde buscadores por labores de SEO y de SEM, etc.  

Dichos usuarios tienen acceso a diferentes productos de menor contenido a cambio de registrarse en un formulario del site, en el que cumplimentan datos personales, email, teléfono, profesión, etc. El usuario pasa a ser Registrado, y es lo que en el mundo del ecommerce se denomina “lead”.  

Tras el registro se permite el acceso a información muy básica sobre las empresas buscadas (Ficha de empresa), y se otorga la posibilidad de consumir gratis 5 productos de información a los que llamamos perfiles de empresa.  

La oferta caduca a los 30 días.  

El Perfil de empresa es un producto con contenido básico y su objetivo es mostrar a los potenciales compradores el nivel del contenido de los productos que suministra el ecommerce.  

Resumiendo, la dinámica es la siguiente:
* Un usuario busca una empresa en internet o en la web.
* Al seleccionar una de las listadas en la búsqueda, se presenta una Ficha (Identificación) y se crea un registro en el log. Se muestra los productos disponibles para consumir.  
    * Productos Promocionales (Perfil de Empresa). Para usuarios Registrados, si el Usuario se registra.
    * Resto de Productos son de Pago. Es necesario estar registrado y pagar.
* El usuario consume un producto (registro en el log) o vuelve a buscar otra empresa.  

Si el usuario Registrado está interesado en conocer más a fondo una empresa o un determinado producto de pago, se le ofrece la posibilidad de contratación:  
* PPV: Compra puntual del producto
* Bonos: compra de un conjunto de unidades o una cantidad de productos a cambio de un pago anticipado.
* Suscripción: pagando una cantidad periodica permite el acceso y consumo de productos, limitado por el volumen de compra y por la fecha de caducidad de la suscripción

Cuando se produce una de estas contrataciones el usuario “Registrado”, pasa a ser “Cliente”.

El propietario del ecommerce tiene información del usuario de su plataforma de Google Analytics que utiliza para la captación en internet, uso de cookies, estrategias de SEO y SEM, acuerdos con portales.  

Ahora desea dar un paso más y quiere conocer más sobre los usuarios registrados para determinar la probabilidad de conversión a cliente. Saber cuáles son potenciales compradores a partir de los datos de ese registro en sus sistemas, del hábito de consumo de productos, y del tipo de empresa buscada/consultada.  

El conjunto de datos está muy desbalanceado por ello es muy importante determinar a qué usuarios registrados debe dirigir el ecommerce sus campañas de captación.  

Por otra parte, pero relacionado con el mismo tema, desean saber que perfil tienen sus clientes y que variables son las mas influyentes/relevantes en la predicción de compra para modificar la estrategia de marketing y su posicionamiento en Internet.  


------------------------------------

### Librerias

In [1]:
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
# pd.options.display.max_columns

------------------------------------

### Lectura de Ficheros

A continuación comenzamos leyendo los ficheros que hemos tratado, limpiado y analizado en los procesos anteriores.

In [2]:
df_cons = pd.read_csv('Data/Consumos_Clean.txt', sep = ';', encoding = 'ISO-8859-1')

In [3]:
df_cons.shape

(406346, 20)

In [4]:
df_cons.dtypes

ID_USUARIO                  int64
CONS_FECHACONSUMO          object
FICH_PROMO_TOT              int64
PERF_PROMO_TOT              int64
EST_ACTIVA_TOT            float64
EST_INACTIVA_TOT          float64
EST_REESTRUCT_TOT         float64
EST_CLINTON_TOT           float64
EST_NI_TOT                float64
SECT_PRIM_TOT             float64
SECT_SECU_TOT             float64
SECT_TERC_TOT             float64
SECT_CUAT_TOT             float64
SECT_QUIN_TOT             float64
SECT_SD_TOT               float64
1A_DESCGRUPOPROD           object
1A_EMPCONSUL_EST           object
1A_EMPCONSUL_SECTOR        object
1A_EMPCONSUL_PROV_DENS    float64
1A_EMPCONSUL_PROV_IDH     float64
dtype: object

In [5]:
df_user = pd.read_csv('Data/Usuarios_Clean.txt', sep = ';', encoding = 'ISO-8859-1')

In [6]:
df_user.shape

(367637, 25)

In [7]:
df_user.dtypes

ID_USUARIO               int64
TIPOUSUARIO             object
FECHA_REGISTRO          object
CANAL_REGISTRO           int64
FECHA_ALTA              object
IND_ALTA                 int64
FECHA_CLIENTE           object
IND_CLIENTE              int64
TIPOEMAIL               object
BONDAD_EMAIL             int64
TIPO_TELF               object
IPCASOS                  int64
IP_COUNTRY              object
IP_REGION               object
IP_COUNTRY_PIB          object
IP_COUNTRY_IDH_NUM      object
IP_COUNTRY_IDH_GROUP    object
USU_TIPO                object
USU_TAMANIO             object
USU_ESTADO              object
USU_SECCION_DESC        object
USU_SECTOR_ECONOMICO    object
USU_DEPARTAMENTO        object
USU_DEPART_DEN_POB      object
USU_DEPART_IDH          object
dtype: object

--------------------------

### Unificamos la información

Añadimos a la iformación de cada cliente, su respectiva información de cosumo

In [8]:
df = pd.merge(left = df_user, right = df_cons, how = "inner", left_on = "ID_USUARIO", right_on = "ID_USUARIO")

In [9]:
df.head(5)

,ID_USUARIO,TIPOUSUARIO,FECHA_REGISTRO,CANAL_REGISTRO,FECHA_ALTA,IND_ALTA,FECHA_CLIENTE,IND_CLIENTE,TIPOEMAIL,BONDAD_EMAIL,TIPO_TELF,IPCASOS,IP_COUNTRY,IP_REGION,IP_COUNTRY_PIB,IP_COUNTRY_IDH_NUM,IP_COUNTRY_IDH_GROUP,USU_TIPO,USU_TAMANIO,USU_ESTADO,USU_SECCION_DESC,USU_SECTOR_ECONOMICO,USU_DEPARTAMENTO,USU_DEPART_DEN_POB,USU_DEPART_IDH,CONS_FECHACONSUMO,FICH_PROMO_TOT,PERF_PROMO_TOT,EST_ACTIVA_TOT,EST_INACTIVA_TOT,EST_REESTRUCT_TOT,EST_CLINTON_TOT,EST_NI_TOT,SECT_PRIM_TOT,SECT_SECU_TOT,SECT_TERC_TOT,SECT_CUAT_TOT,SECT_QUIN_TOT,SECT_SD_TOT,1A_DESCGRUPOPROD,1A_EMPCONSUL_EST,1A_EMPCONSUL_SECTOR,1A_EMPCONSUL_PROV_DENS,1A_EMPCONSUL_PROV_IDH
0,8107310,PF,2019-10-22,3,NoApl,0,NoApl,0,GOOHOTYAHMAIL,0,NO_PUBLICO,1,COLOMBIA,Valle del Cauca,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,2019-10-22,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,PERF_PROMO,ACTIVA,TERCIARIO,47.92,0.782
1,7784565,PJ,2019-05-14,3,NoApl,0,NoApl,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Antioquia,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,PQ,ACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,QUINDIO,306.38,0.832,2019-05-14,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,PERF_PROMO,ACTIVA,SD,47.92,0.782
2,7718778,PJ,2019-09-04,7,NoApl,0,NoApl,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Bolivar,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,MC,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,ATLANTICO,815.17,0.835,2019-09-04,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,PERF_PROMO,ACTIVA,TERCIARIO,4438.75,0.904
3,7952765,PF,2019-12-08,3,NoApl,0,NoApl,0,DOCENTE,1,NO_PUBLICO,1,COLOMBIA,Risaralda,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,2019-12-08,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,PERF_PROMO,ACTIVA,CUATERNARIO,229.94,0.839
4,7855424,PJ,2019-06-21,7,NoApl,0,NoApl,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Atlántico,Ingreso mediano alto,0.767,Alto,EMPRESARIO INDIVIDUAL,MC,INACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,ATLANTICO,815.17,0.835,2019-06-21,0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,PERF_PROMO,ACTIVA,SECUNDARIO,815.17,0.835


In [10]:
df.isna().sum()

ID_USUARIO                0
TIPOUSUARIO               0
FECHA_REGISTRO            0
CANAL_REGISTRO            0
FECHA_ALTA                0
IND_ALTA                  0
FECHA_CLIENTE             0
IND_CLIENTE               0
TIPOEMAIL                 0
BONDAD_EMAIL              0
TIPO_TELF                 0
IPCASOS                   0
IP_COUNTRY                0
IP_REGION                 0
IP_COUNTRY_PIB            0
IP_COUNTRY_IDH_NUM        0
IP_COUNTRY_IDH_GROUP      0
USU_TIPO                  0
USU_TAMANIO               0
USU_ESTADO                0
USU_SECCION_DESC          0
USU_SECTOR_ECONOMICO      0
USU_DEPARTAMENTO          0
USU_DEPART_DEN_POB        0
USU_DEPART_IDH            0
CONS_FECHACONSUMO         0
FICH_PROMO_TOT            0
PERF_PROMO_TOT            0
EST_ACTIVA_TOT            1
EST_INACTIVA_TOT          1
EST_REESTRUCT_TOT         1
EST_CLINTON_TOT           1
EST_NI_TOT                1
SECT_PRIM_TOT             1
SECT_SECU_TOT             1
SECT_TERC_TOT       

**Eliminamos los registros nulos**

In [11]:
df.dropna(axis = 0, inplace = True)

-------------------

### Creación de nuevas variables

En este apartado trabajaremos con las variables del dataframe de forma que se tengan nuevas variables.

**1. Extración de información sobre Fechas**

Tenemos un total de hasta 4 Fechas en el dataset:
* FECHA_REGISTRO
* FECHA_ALTA
* FECHA_CLIENTE
* 1A_CONS_FECHACONSUMO

De cada variable indicada se generará la siguiente información:
- `weekday`: Variable categórica con el día de la semana

**Transformacion de weekday**

In [12]:
df.loc[df['FECHA_REGISTRO'] == 'NoApl', 'FECHA_REGISTRO'] = None
df['FECHA_REGISTRO'] = pd.to_datetime(df['FECHA_REGISTRO'])
df['FECHA_REGISTRO_WEEKDAY'] = df['FECHA_REGISTRO'].dt.weekday
df['FECHA_REGISTRO_WEEKDAY'] = df['FECHA_REGISTRO_WEEKDAY'].fillna(-1)
df['FECHA_REGISTRO_WEEKDAY'] = df['FECHA_REGISTRO_WEEKDAY'].astype('int')

In [13]:
df['FECHA_REGISTRO_WEEKDAY'].value_counts()

2    69786
3    69660
1    67480
0    65458
4    57607
5    41381
6    34895
Name: FECHA_REGISTRO_WEEKDAY, dtype: int64

In [14]:
df.loc[df['FECHA_ALTA'] == 'NoApl', 'FECHA_ALTA'] = None
df['FECHA_ALTA'] = pd.to_datetime(df['FECHA_ALTA'])
df['FECHA_ALTA_WEEKDAY'] = df['FECHA_ALTA'].dt.weekday
df['FECHA_ALTA_WEEKDAY'] = df['FECHA_ALTA_WEEKDAY'].fillna(-1)
df['FECHA_ALTA_WEEKDAY'] = df['FECHA_ALTA_WEEKDAY'].astype('int')

In [15]:
df['FECHA_ALTA_WEEKDAY'].value_counts()

-1    393608
 3      2893
 2      2202
 1      2109
 0      1991
 4      1735
 6       903
 5       826
Name: FECHA_ALTA_WEEKDAY, dtype: int64

In [16]:
df.loc[df['FECHA_CLIENTE'] == 'NoApl', 'FECHA_CLIENTE'] = None
df['FECHA_CLIENTE'] = pd.to_datetime(df['FECHA_CLIENTE'])
df['FECHA_CLIENTE_WEEKDAY'] = df['FECHA_CLIENTE'].dt.weekday
df['FECHA_CLIENTE_WEEKDAY'] = df['FECHA_CLIENTE_WEEKDAY'].fillna(-1)
df['FECHA_CLIENTE_WEEKDAY'] = df['FECHA_CLIENTE_WEEKDAY'].astype('int')

In [17]:
df['FECHA_CLIENTE_WEEKDAY'].value_counts()

-1    395427
 3      2566
 1      1893
 2      1842
 0      1610
 4      1462
 6       770
 5       697
Name: FECHA_CLIENTE_WEEKDAY, dtype: int64

In [18]:
df.loc[df['CONS_FECHACONSUMO'] == 'NoApl', 'CONS_FECHACONSUMO'] = None
df['CONS_FECHACONSUMO'] = pd.to_datetime(df['CONS_FECHACONSUMO'])
df['FECHACONSUMO_WEEKDAY'] = df['CONS_FECHACONSUMO'].dt.weekday
df['FECHACONSUMO_WEEKDAY'] = df['FECHACONSUMO_WEEKDAY'].fillna(-1)
df['FECHACONSUMO_WEEKDAY'] = df['FECHACONSUMO_WEEKDAY'].astype('int')

In [19]:
df['FECHACONSUMO_WEEKDAY'].value_counts()

2    70078
3    69128
1    67418
0    65636
4    57742
5    41443
6    34822
Name: FECHACONSUMO_WEEKDAY, dtype: int64

**2. IP_CONTINENTE**

Se creará una variable que agrupe por continente cada uno de los paises de la variable IP_COUNTRY

In [20]:
df['IP_COUNTRY'].unique()

array(['COLOMBIA', 'NI', 'ESTADOS UNIDOS', 'SUIZA', 'PERU', 'REINO UNIDO',
       'VENEZUELA', 'CHILE', 'ECUADOR', 'JAPON', 'ARGENTINA', 'PANAMA',
       'FRANCIA', 'CAMERUN', 'ESPAÑA', 'CANADA', 'BRASIL', 'ESLOVENIA',
       'ALEMANIA', 'MEXICO', 'AUSTRIA', 'PAISES BAJOS', 'URUGUAY',
       'AUSTRALIA', 'COSTA RICA', 'SUECIA', 'ITALIA', 'FINLANDIA',
       'CHINA', 'RUSIA', 'BOLIVIA', 'GUATEMALA', 'PARAGUAY', 'POLONIA',
       'MALTA', 'EMIRATOS ARABES UNIDOS', 'SINGAPUR', 'PUERTO RICO',
       'INDIA', 'BELGICA', 'ARUBA', 'HONG KONG', 'ANDORRA',
       'REPUBLICA DOMINICANA', 'NORUEGA', 'ISRAEL', 'BELICE', 'JERSEY',
       'REPUBLICA CHECA', 'NICARAGUA', 'CURACAO', 'BONAIRE', 'TAIWAN',
       'TURQUIA', 'COREA', 'VIETNAM', 'CUBA', 'ARABIA SAUDITA', 'KUWAIT',
       'QATAR', 'GUINEA ECUATORIAL', 'NUEVA ZELANDA', 'DINAMARCA',
       'RUMANIA', 'HONDURAS', 'PORTUGAL', 'ANGOLA', 'UCRANIA', 'NIGERIA',
       'CROACIA', 'TAILANDIA', 'LUXEMBURGO', 'IRLANDA', 'GUADALUPE',
       'EL SALVADOR

In [21]:
df['IP_CONTINENTE'] = None

In [22]:
# EUROPA
df.loc[(df['IP_COUNTRY'].isin(['SUIZA','REINO UNIDO','FRANCIA','ESPAÑA','ESLOVENIA','ALEMANIA','AUSTRIA','PAISES BAJOS',
                               'SUECIA','ITALIA','FINLANDIA','RUSIA','POLONIA','MALTA','BELGICA','ANDORRA','NORUEGA',
                               'JERSEY','REPUBLICA CHECA','DINAMARCA','RUMANIA','PORTUGAL','UCRANIA','CROACIA','LUXEMBURGO', 
                               'IRLANDA','MOLDAVIA'])), 'IP_CONTINENTE'] = 'EUROPA'

# ASIA
df.loc[(df['IP_COUNTRY'].isin(['JAPON','CHINA','EMIRATOS ARABES UNIDOS','SINGAPUR','INDIA','HONG KONG','ISRAEL',
                               'TAIWAN','BAHREIN','TAILANDIA','COREA','TURQUIA','VIETNAM','ARABIA SAUDITA', 
                               'KUWAIT','QATAR'])), 'IP_CONTINENTE'] = 'ASIA'

# AFRICA
df.loc[(df['IP_COUNTRY'].isin(['CAMERUN','GUINEA ECUATORIAL','ANGOLA','NIGERIA'])), 'IP_CONTINENTE'] = 'AFRICA'

# OCEANNIA
df.loc[(df['IP_COUNTRY'].isin(['AUSTRALIA','NUEVA ZELANDA'])), 'IP_CONTINENTE'] = 'OCEANNIA'

# AMERICA DEL SUR
df.loc[(df['IP_COUNTRY'].isin(['COLOMBIA','PERU','VENEZUELA','CHILE','ECUADOR','ARGENTINA','BRASIL','URUGUAY',
                               'BOLIVIA','PARAGUAY','ARUBA','CURACAO','BONAIRE'])), 'IP_CONTINENTE'] = 'AMERICA_SUR'

# AMERICA DEL NORTE
df.loc[(df['IP_COUNTRY'].isin(['ESTADOS UNIDOS','PANAMA','CANADA','MEXICO','COSTA RICA','GUATEMALA','PUERTO RICO',
                               'REPUBLICA DOMINICANA','BELICE','NICARAGUA','BARBADOS','CUBA','HONDURAS','GUADALUPE',
                               'EL SALVADOR'])), 'IP_CONTINENTE'] = 'AMERICA_NORTE'

# NO IDENTIFICADO
df.loc[(df['IP_COUNTRY'].isin(['NI'])), 'IP_CONTINENTE'] = 'NI'

In [23]:
df['IP_CONTINENTE'].value_counts()

AMERICA_SUR      372210
NI                29638
AMERICA_NORTE      3520
EUROPA              662
ASIA                193
OCEANNIA             40
AFRICA                4
Name: IP_CONTINENTE, dtype: int64

In [24]:
df['IP_CONTINENTE'].unique()

array(['AMERICA_SUR', 'NI', 'AMERICA_NORTE', 'EUROPA', 'ASIA', 'AFRICA',
       'OCEANNIA'], dtype=object)

**3. IP_COLOMBIA**

La siguiente variable nos informa de si la ip del registro se realiza desde colombia (mismo lugar perteneciente al ecommerce) o no
* 1: La IP es desde Colombia
* 0: Caso Contrario

In [25]:
df['IP_COLOMBIA'] = None

In [26]:
df.loc[(df['IP_COUNTRY'].isin(['COLOMBIA'])), 'IP_COLOMBIA'] = 1
df.loc[(~df['IP_COUNTRY'].isin(['COLOMBIA'])), 'IP_COLOMBIA'] = 0

In [27]:
df['IP_COLOMBIA'].value_counts()

1    370271
0     35996
Name: IP_COLOMBIA, dtype: int64

**4. meses de las fechas**

In [28]:
df.loc[df['FECHA_REGISTRO'] == 'NoApl', 'FECHA_REGISTRO'] = None
df['FECHA_REGISTRO'] = pd.to_datetime(df['FECHA_REGISTRO'])
df['FECHA_REGISTRO_MONTH'] = df['FECHA_REGISTRO'].dt.month
df['FECHA_REGISTRO_MONTH'] = df['FECHA_REGISTRO_MONTH'].fillna(-1)
df['FECHA_REGISTRO_MONTH'] = df['FECHA_REGISTRO_MONTH'].astype('int')

In [29]:
df['FECHA_REGISTRO_MONTH'].value_counts()

5     42089
8     36139
10    35596
11    34876
7     34763
4     34720
3     34613
9     33923
2     33109
6     31119
1     28084
12    27236
Name: FECHA_REGISTRO_MONTH, dtype: int64

In [30]:
df.loc[df['FECHA_ALTA'] == 'NoApl', 'FECHA_ALTA'] = None
df['FECHA_ALTA'] = pd.to_datetime(df['FECHA_ALTA'])
df['FECHA_ALTA_MONTH'] = df['FECHA_ALTA'].dt.month
df['FECHA_ALTA_MONTH'] = df['FECHA_ALTA_MONTH'].fillna(-1)
df['FECHA_ALTA_MONTH'] = df['FECHA_ALTA_MONTH'].astype('int')

In [31]:
df['FECHA_ALTA_MONTH'].value_counts()

-1     393608
 8       1555
 11      1454
 5       1319
 1       1009
 2        983
 7        973
 6        969
 12       963
 9        951
 10       929
 4        882
 3        672
Name: FECHA_ALTA_MONTH, dtype: int64

In [32]:
df.loc[df['FECHA_CLIENTE'] == 'NoApl', 'FECHA_CLIENTE'] = None
df['FECHA_CLIENTE'] = pd.to_datetime(df['FECHA_CLIENTE'])
df['FECHA_CLIENTE_MONTH'] = df['FECHA_CLIENTE'].dt.month
df['FECHA_CLIENTE_MONTH'] = df['FECHA_CLIENTE_MONTH'].fillna(-1)
df['FECHA_CLIENTE_MONTH'] = df['FECHA_CLIENTE_MONTH'].astype('int')

In [33]:
df['FECHA_CLIENTE_MONTH'].value_counts()

-1     395427
 8       1384
 11      1274
 5       1057
 2        892
 1        886
 12       868
 9        865
 6        826
 7        812
 10       774
 4        702
 3        500
Name: FECHA_CLIENTE_MONTH, dtype: int64

In [34]:
df.loc[df['CONS_FECHACONSUMO'] == 'NoApl', 'CONS_FECHACONSUMO'] = None
df['CONS_FECHACONSUMO'] = pd.to_datetime(df['CONS_FECHACONSUMO'])
df['FECHACONSUMO_MONTH'] = df['CONS_FECHACONSUMO'].dt.month
df['FECHACONSUMO_MONTH'] = df['FECHACONSUMO_MONTH'].fillna(-1)
df['FECHACONSUMO_MONTH'] = df['FECHACONSUMO_MONTH'].astype('int')

In [35]:
df['FECHACONSUMO_MONTH'].value_counts()

5     41523
8     35814
11    35586
10    35530
7     34980
4     34729
3     34328
9     34323
2     32676
6     31365
1     28241
12    27172
Name: FECHACONSUMO_MONTH, dtype: int64

**5. Hay variables categóricas que deben convertirse en numéricas**

5.1 IP_COUNTRY_IDH_NUM

In [36]:
df['IP_COUNTRY_IDH_NUM'].unique()

array(['0.767', 'NI', '0.926', '0.955', '0.778', '0.932', '0.711',
       '0.851', '0.759', '0.919', '0.845', '0.815', '0.901', '0.563',
       '0.904', '0.929', '0.765', '0.917', '0.947', '0.779', '0.922',
       '0.944', '0.817', '0.81', '0.945', '0.892', '0.938', '0.761',
       '0.824', '0.718', '0.663', '0.728', '0.88', '0.895', '0.89',
       '0.645', '0.931', '0.949', '0.868', '0.756', '0.957', '0.716',
       '0.9', '0.66', '0.82', '0.916', '0.704', '0.783', '0.854', '0.806',
       '0.848', '0.592', '0.94', '0.828', '0.682', '0.864', '0.581',
       '0.539', '0.777', '0.673', '0.75', '0.814', '0.852'], dtype=object)

In [37]:
df.loc[df['IP_COUNTRY_IDH_NUM'] == 'NI', 'IP_COUNTRY_IDH_NUM'] = 0

In [38]:
df['IP_COUNTRY_IDH_NUM'] = df['IP_COUNTRY_IDH_NUM'].astype('float')

In [39]:
df['IP_COUNTRY_IDH_NUM'].unique()

array([0.767, 0.   , 0.926, 0.955, 0.778, 0.932, 0.711, 0.851, 0.759,
       0.919, 0.845, 0.815, 0.901, 0.563, 0.904, 0.929, 0.765, 0.917,
       0.947, 0.779, 0.922, 0.944, 0.817, 0.81 , 0.945, 0.892, 0.938,
       0.761, 0.824, 0.718, 0.663, 0.728, 0.88 , 0.895, 0.89 , 0.645,
       0.931, 0.949, 0.868, 0.756, 0.957, 0.716, 0.9  , 0.66 , 0.82 ,
       0.916, 0.704, 0.783, 0.854, 0.806, 0.848, 0.592, 0.94 , 0.828,
       0.682, 0.864, 0.581, 0.539, 0.777, 0.673, 0.75 , 0.814, 0.852])

5.2 USU_DEPART_DEN_POB

In [40]:
df['USU_DEPART_DEN_POB'].unique()

array(['NOAPL', '306.38', '815.17', '207.86', '67.5', '229.94', '101.49',
       '4438.75', '45.94', '44.92', '110.7', '62.63', '47.92', '55.47',
       '68.33', '59.77', '125.25', '80.73', '54.32', '11.23', '58.06',
       '79.81', '8.01', 'SD', '53.15', '11.01', '10.75', '5.37', '0.68',
       '13.87', '1410.0', '0.679', '2.04', '0.56', '0.79'], dtype=object)

In [41]:
df.loc[df['USU_DEPART_DEN_POB'] == 'NOAPL', 'USU_DEPART_DEN_POB'] = -1
df.loc[df['USU_DEPART_DEN_POB'] == 'SD', 'USU_DEPART_DEN_POB'] = 0

In [42]:
df['USU_DEPART_DEN_POB'] = df['USU_DEPART_DEN_POB'].astype('float')

In [43]:
df['USU_DEPART_DEN_POB'].unique()

array([-1.00000e+00,  3.06380e+02,  8.15170e+02,  2.07860e+02,
        6.75000e+01,  2.29940e+02,  1.01490e+02,  4.43875e+03,
        4.59400e+01,  4.49200e+01,  1.10700e+02,  6.26300e+01,
        4.79200e+01,  5.54700e+01,  6.83300e+01,  5.97700e+01,
        1.25250e+02,  8.07300e+01,  5.43200e+01,  1.12300e+01,
        5.80600e+01,  7.98100e+01,  8.01000e+00,  0.00000e+00,
        5.31500e+01,  1.10100e+01,  1.07500e+01,  5.37000e+00,
        6.80000e-01,  1.38700e+01,  1.41000e+03,  6.79000e-01,
        2.04000e+00,  5.60000e-01,  7.90000e-01])

5.3 USU_DEPART_IDH

In [44]:
df['USU_DEPART_IDH'].unique()

array(['NOAPL', '0.832', '0.835', '0.861', '0.879', '0.839', '0.849',
       '0.904', '0.691', '0.81', '0.837', '0.799', '0.782', '0.842',
       '0.798', '0.804', '0.828', '0.823', '0.785', '0.822', '0.807',
       '0.775', '0.867', 'SD', '0.773', '0.731', '0.752', '0.768',
       '0.759', '0.834'], dtype=object)

In [45]:
df.loc[df['USU_DEPART_IDH'] == 'NOAPL', 'USU_DEPART_IDH'] = -1
df.loc[df['USU_DEPART_IDH'] == 'SD', 'USU_DEPART_IDH'] = 0

In [46]:
df['USU_DEPART_IDH'] = df['USU_DEPART_IDH'].astype('float')

In [47]:
df['USU_DEPART_IDH'].unique()

array([-1.   ,  0.832,  0.835,  0.861,  0.879,  0.839,  0.849,  0.904,
        0.691,  0.81 ,  0.837,  0.799,  0.782,  0.842,  0.798,  0.804,
        0.828,  0.823,  0.785,  0.822,  0.807,  0.775,  0.867,  0.   ,
        0.773,  0.731,  0.752,  0.768,  0.759,  0.834])

------------

------------

### Reducción de dimensionalidad

Por último eliminamos columnas, renombramos y ordenamos al gusto

**Debido a la gran cantidad de categorías que tienen y que se han generado nuevas variables de forma que se explicaban estas, omitiremos las siguientes variables**:
* IP_COUNTRY
* IP_REGION
* USU_SECCION_DESC
* 1A_CONS_EMPCONSUL_SECCION_DESC
* 1A_CONS_DESCPRODUCTO
* 1A_CONS_EMPCONSUL_PROV
* USU_DEPARTAMENTO

In [49]:
df = df[['TIPOUSUARIO','FECHA_REGISTRO_WEEKDAY','FECHA_REGISTRO_MONTH','CANAL_REGISTRO','FECHA_ALTA_WEEKDAY','FECHA_ALTA_MONTH',
         'IND_ALTA','FECHA_CLIENTE_WEEKDAY','FECHA_CLIENTE_MONTH','IND_CLIENTE','TIPOEMAIL','BONDAD_EMAIL',
         'TIPO_TELF','IPCASOS','IP_CONTINENTE','IP_COLOMBIA','IP_COUNTRY_PIB','IP_COUNTRY_IDH_NUM','IP_COUNTRY_IDH_GROUP','USU_TIPO','USU_TAMANIO','USU_ESTADO','USU_SECTOR_ECONOMICO',
         'USU_DEPART_DEN_POB','USU_DEPART_IDH','FECHACONSUMO_WEEKDAY','FECHACONSUMO_MONTH','1A_DESCGRUPOPROD','1A_EMPCONSUL_EST',
         '1A_EMPCONSUL_SECTOR','1A_EMPCONSUL_PROV_DENS','1A_EMPCONSUL_PROV_IDH','FICH_PROMO_TOT','PERF_PROMO_TOT','EST_ACTIVA_TOT','EST_INACTIVA_TOT','EST_REESTRUCT_TOT',
         'EST_CLINTON_TOT','EST_NI_TOT','SECT_PRIM_TOT','SECT_SECU_TOT','SECT_TERC_TOT','SECT_CUAT_TOT','SECT_QUIN_TOT','SECT_SD_TOT']]

In [50]:
df.head(3)

,TIPOUSUARIO,FECHA_REGISTRO_WEEKDAY,FECHA_REGISTRO_MONTH,CANAL_REGISTRO,FECHA_ALTA_WEEKDAY,FECHA_ALTA_MONTH,IND_ALTA,FECHA_CLIENTE_WEEKDAY,FECHA_CLIENTE_MONTH,IND_CLIENTE,TIPOEMAIL,BONDAD_EMAIL,TIPO_TELF,IPCASOS,IP_CONTINENTE,IP_COLOMBIA,IP_COUNTRY_PIB,IP_COUNTRY_IDH_NUM,IP_COUNTRY_IDH_GROUP,USU_TIPO,USU_TAMANIO,USU_ESTADO,USU_SECTOR_ECONOMICO,USU_DEPART_DEN_POB,USU_DEPART_IDH,FECHACONSUMO_WEEKDAY,FECHACONSUMO_MONTH,1A_DESCGRUPOPROD,1A_EMPCONSUL_EST,1A_EMPCONSUL_SECTOR,1A_EMPCONSUL_PROV_DENS,1A_EMPCONSUL_PROV_IDH,FICH_PROMO_TOT,PERF_PROMO_TOT,EST_ACTIVA_TOT,EST_INACTIVA_TOT,EST_REESTRUCT_TOT,EST_CLINTON_TOT,EST_NI_TOT,SECT_PRIM_TOT,SECT_SECU_TOT,SECT_TERC_TOT,SECT_CUAT_TOT,SECT_QUIN_TOT,SECT_SD_TOT
0,PF,1,10,3,-1,-1,0,-1,-1,0,GOOHOTYAHMAIL,0,NO_PUBLICO,1,AMERICA_SUR,1,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,-1.00,-1.000,1,10,PERF_PROMO,ACTIVA,TERCIARIO,47.92,0.782,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,PJ,1,5,3,-1,-1,0,-1,-1,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,AMERICA_SUR,1,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,PQ,ACTIVA,TERCIARIO,306.38,0.832,1,5,PERF_PROMO,ACTIVA,SD,47.92,0.782,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,PJ,2,9,7,-1,-1,0,-1,-1,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,AMERICA_SUR,1,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,MC,ACTIVA,TERCIARIO,815.17,0.835,2,9,PERF_PROMO,ACTIVA,TERCIARIO,4438.75,0.904,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [51]:
df.shape

(406267, 45)

---------------

---------------

In [52]:
df.isna().sum()

TIPOUSUARIO               0
FECHA_REGISTRO_WEEKDAY    0
FECHA_REGISTRO_MONTH      0
CANAL_REGISTRO            0
FECHA_ALTA_WEEKDAY        0
FECHA_ALTA_MONTH          0
IND_ALTA                  0
FECHA_CLIENTE_WEEKDAY     0
FECHA_CLIENTE_MONTH       0
IND_CLIENTE               0
TIPOEMAIL                 0
BONDAD_EMAIL              0
TIPO_TELF                 0
IPCASOS                   0
IP_CONTINENTE             0
IP_COLOMBIA               0
IP_COUNTRY_PIB            0
IP_COUNTRY_IDH_NUM        0
IP_COUNTRY_IDH_GROUP      0
USU_TIPO                  0
USU_TAMANIO               0
USU_ESTADO                0
USU_SECTOR_ECONOMICO      0
USU_DEPART_DEN_POB        0
USU_DEPART_IDH            0
FECHACONSUMO_WEEKDAY      0
FECHACONSUMO_MONTH        0
1A_DESCGRUPOPROD          0
1A_EMPCONSUL_EST          0
1A_EMPCONSUL_SECTOR       0
1A_EMPCONSUL_PROV_DENS    0
1A_EMPCONSUL_PROV_IDH     0
FICH_PROMO_TOT            0
PERF_PROMO_TOT            0
EST_ACTIVA_TOT            0
EST_INACTIVA_TOT    

### Escritura de Datos

In [53]:
df.to_csv('Data/dataset_model.txt', sep = ';', encoding = 'ISO-8859-1', index = False)